<a href="https://colab.research.google.com/github/christiejibaraki/CUREBench/blob/main/notebooks/0-colab-environment-setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run CUREBench starter code with gpt oss model
- OpenAI GPT-OSS-20B open-weight reasoning model trained with Harmony response format
  - 20.9B params, 3.6B active per token
  - Mixture-of-Experts (MoE) transformer
  - `bitsandbytes` enables 8-bit quantization so GPT-OSS-20B can run on 16GB GPU

In [1]:
# tooklkit version
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [2]:
import os
import sys
import os.path

In [3]:
os.environ['BNB_CUDA_VERSION'] = '125'
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

## Setup
- Clone forked CUREBench repo onto local `content` folder (this is not persistent)
- Create virtual environment and install packages
  - Package installation takes about **5 min** ⏰
- Add virtual environment's site-packages to notebook's system path

In [4]:
!git clone https://github.com/christiejibaraki/CUREBench.git

Cloning into 'CUREBench'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 111 (delta 48), reused 34 (delta 17), pack-reused 16 (from 1)
Receiving objects: 100% (111/111), 2.48 MiB | 23.53 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [5]:
%cd CUREBench

/content/CUREBench


In [6]:
# !git pull

In [6]:
!python --version

Python 3.12.12


In [7]:
!pip install virtualenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 42.5 MB/s eta 0:00:00


In [8]:
!virtualenv env

created virtual environment CPython3.12.12.final.0-64 in 329ms
  creator CPython3Posix(dest=/content/CUREBench/env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==25.2
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [9]:
!./env/bin/pip list

Package Version
------- -------
pip     25.2


In [10]:
!./env/bin/pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 123.8 MB/s  0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.4/567.4 kB 28.4 MB/s  0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 148.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 108.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 187.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 71.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 20.9 MB/s  0:00:22
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 69.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.

In [11]:
env_path = './env/lib/python3.12/site-packages'
sys.path.append(env_path)

## GPT OSS Tutorial from CUREBench
- loading the model will take about **5 min** ⏰

In [12]:
from core.eval_framework import GPTOSS20BModel
from tooluniverse.execute_function import ToolUniverse
from openai_harmony import load_harmony_encoding

<frozen importlib._bootstrap>:488: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterHierarchyMatcher> already registered; second conversion method ignored.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/hyperopt/atpe.py:19: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow

In [15]:
#make sure encodings are present
load_harmony_encoding('HarmonyGptOss')

In [16]:
model = GPTOSS20BModel("openai/gpt-oss-20b", reasoning_lvl="low", quantization="auto",
                       system_identity="You are a clinical trial assistant.")
model.load()

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

testing.py: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Fetching 41 files:   0%|          | 0/41 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [18]:
# final, trace = model.inference("What’s 45 * 67?", builtin_tools=["python"])

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [17]:
def convert_tooluniverse_to_openai(tools):
    """Convert ToolUniverse JSON tools into OpenAI-compatible schemas."""
    converted = []
    for t in tools:
        param = t.get("parameter", {"type": "object", "properties": {}})
        props = param.get("properties") or {}  # <-- ensure dict, not None

        # Collect required fields from the ToolUniverse format
        required_fields = [k for k, v in props.items() if v.get("required", False)]

        converted.append({
            "type": "function",
            "function": {
                "name": t.get("name"),
                "description": t.get("description", ""),
                "parameters": {
                    "type": "object",
                    "properties": {
                        k: {kk: vv for kk, vv in v.items() if kk != "required"}
                        for k, v in props.items()
                    },
                    "required": required_fields,
                },
            },
        })
    return converted

In [18]:
def test_dryrun_tooluniverse():
    """Test 1: External ToolUniverse (dry-run only, safe everywhere)."""
    print("=== Test 1: Dry-run with ToolUniverse ===")
    model = GPTOSS20BModel(model_name="openai/gpt-oss-20b", quantization="auto")
    model.load()

    engine = ToolUniverse()
    engine.load_tools()
    raw_tools = engine.return_all_loaded_tools()
    tools = convert_tooluniverse_to_openai(raw_tools)
    print(f"✓ Loaded {len(tools)} ToolUniverse tools")

    messages = [
        {"role": "system", "content": "You are a helpful assistant with access to tools."},
        {"role": "user", "content": "What are the active ingredients in Panadol?"},
    ]

    input_ids = model.tokenizer.apply_chat_template(
        messages,
        tools=tools,
        add_generation_prompt=True,
        return_tensors="pt",
    )

    print("Dry-run successful")
    print(f"Input IDs shape: {tuple(input_ids.shape)}")
    print(f"First 20 token IDs: {input_ids[0][:20].tolist()}")

In [19]:
test_dryrun_tooluniverse()

=== Test 1: Dry-run with ToolUniverse ===


Fetching 41 files:   0%|          | 0/41 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Fetching 41 files:   0%|          | 0/41 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 254.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 20.12 MiB is free. Process 3205 has 14.72 GiB memory in use. Of the allocated memory 14.54 GiB is allocated by PyTorch, and 81.22 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)